# Custom PPO Training: Qwen 0.6B on Rule-Based Task

This notebook implements from-scratch PPO training for Qwen2.5-0.6B on a rule-based task.

**Task**: Environment provides a rule → model generates output → environment provides output as context for next rule.

**Features**:
- Custom PPO implementation (verl-inspired patterns)
- Qwen 0.6B with thinking mode enabled
- Full parameter training (NO LoRA)
- Rule-based task environment
- Baseline and full evaluation
- WandB integration

**Note**: This is a custom PPO implementation, not using verl's trainer directly.

## 🛠️ Setup & Dependencies

In [ ]:
# Check GPU
!nvidia-smi -L || true

import sys
print("Python:", sys.version)

In [ ]:
# Install dependencies (no verl - custom PPO implementation)
!pip install -q torch==2.4.0 torchvision==0.19.0 --index-url https://download.pytorch.org/whl/cu121
!pip install -q flash-attn --no-build-isolation
!pip install -q transformers>=4.45.0 accelerate datasets tokenizers
!pip install -q wandb tensorboard pandas matplotlib tqdm

In [ ]:
import os, json, random, time
from pathlib import Path
from typing import List, Dict, Optional, Any, Tuple
from dataclasses import dataclass, asdict

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModelForCausalLM, AutoTokenizer, get_linear_schedule_with_warmup
import wandb
from tqdm.auto import tqdm

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
assert DEVICE == "cuda", "GPU required for training"

## 🔑 Import API Keys

In [ ]:
# Import keys from keys.py
try:
    from keys import WANDB_API_KEY, HF_TOKEN
    print("✓ Keys imported")
    if WANDB_API_KEY:
        os.environ['WANDB_API_KEY'] = WANDB_API_KEY
    if HF_TOKEN:
        os.environ['HF_TOKEN'] = HF_TOKEN
except ImportError:
    print("⚠ keys.py not found")
    WANDB_API_KEY = ""
    HF_TOKEN = ""

In [ ]:
from huggingface_hub import login

try:
    wandb.login()
    print("✓ WandB login successful")
except Exception as e:
    print(f"⚠ WandB: {e}")

try:
    if os.environ.get('HF_TOKEN'):
        login(token=os.environ['HF_TOKEN'])
        print("✓ HuggingFace login successful")
except Exception as e:
    print(f"⚠ HF: {e}")

## ⚙️ Configuration

In [ ]:
@dataclass
class VerlPPOConfig:
    # Model
    model_name: str = "Qwen/Qwen2.5-0.6B"
    precision: str = "bfloat16"
    use_flash_attn: bool = True
    
    # Training
    num_epochs: int = 3
    num_steps_per_epoch: int = 100
    batch_size: int = 4
    samples_per_prompt: int = 4
    gradient_accumulation_steps: int = 4
    learning_rate: float = 1e-6
    weight_decay: float = 0.01
    max_grad_norm: float = 1.0
    warmup_steps: int = 50
    
    # PPO
    ppo_epochs: int = 4
    clip_range: float = 0.2
    value_loss_coef: float = 0.1
    entropy_coef: float = 0.01
    kl_coef: float = 0.1
    gamma: float = 1.0
    gae_lambda: float = 0.95
    normalize_advantages: bool = True
    
    # Generation
    max_prompt_length: int = 512
    max_new_tokens: int = 512
    temperature: float = 0.7
    top_p: float = 0.9
    top_k: int = 50
    
    # Thinking mode (Qwen-specific)
    enable_thinking: bool = True
    thinking_temperature: float = 0.6
    thinking_top_p: float = 0.95
    parse_thinking: bool = True
    
    # Data
    num_train_samples: int = 1000
    num_val_samples: int = 200
    
    # Logging
    wandb_project: str = "verl-qwen-rule-task"
    wandb_run_name: Optional[str] = None
    log_interval: int = 10
    eval_interval: int = 50
    save_interval: int = 100
    output_dir: str = f"./verl_qwen_run_{int(time.time())}"
    
    seed: int = 42
    
    def __post_init__(self):
        if self.wandb_run_name is None:
            mode = "thinking" if self.enable_thinking else "normal"
            self.wandb_run_name = f"verl_ppo_qwen06b_{mode}_{int(time.time())}"
        os.makedirs(self.output_dir, exist_ok=True)

config = VerlPPOConfig()

print(f"Model: {config.model_name}")
print(f"Thinking mode: {config.enable_thinking}")
print(f"Batch size: {config.batch_size} x {config.samples_per_prompt} = {config.batch_size * config.samples_per_prompt}")
print(f"Output: {config.output_dir}")

with open(os.path.join(config.output_dir, "config.json"), "w") as f:
    json.dump(asdict(config), f, indent=2)

In [ ]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(config.seed)
print(f"✓ Seed: {config.seed}")

## 📦 Load Models

In [ ]:
dtype = torch.bfloat16 if config.precision == "bfloat16" else (torch.float16 if config.precision == "float16" else torch.float32)
print(f"Using dtype: {dtype}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
print(f"✓ Tokenizer loaded (vocab: {len(tokenizer)})")

In [ ]:
policy_model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    torch_dtype=dtype,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="flash_attention_2" if config.use_flash_attn else "eager"
)
policy_model.config.use_cache = False
print(f"✓ Policy model: {sum(p.numel() for p in policy_model.parameters())/1e9:.2f}B params")

In [ ]:
reference_model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    torch_dtype=dtype,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="flash_attention_2" if config.use_flash_attn else "eager"
)
reference_model.eval()
for param in reference_model.parameters():
    param.requires_grad = False
print("✓ Reference model loaded and frozen")

In [ ]:
class ValueHead(nn.Module):
    def __init__(self, hidden_size: int):
        super().__init__()
        self.linear = nn.Linear(hidden_size, 1)
        nn.init.orthogonal_(self.linear.weight, gain=0.01)
        nn.init.constant_(self.linear.bias, 0.0)
    
    def forward(self, hidden_states):
        return self.linear(hidden_states)

value_head = ValueHead(policy_model.config.hidden_size).to(DEVICE).to(dtype)
print(f"✓ Value head (hidden_size={policy_model.config.hidden_size})")

## 🌍 Rule-Based Environment

In [ ]:
CATEGORIES = ["fruits", "vegetables", "colors", "animals", "countries", "cities", "numbers", "programming languages"]

class RuleBasedEnvironment:
    def __init__(self, seed: int = 42):
        self.rng = random.Random(seed)
    
    def generate_rule(self, context: Optional[str] = None) -> str:
        if context is None:
            template = self.rng.choice(["List {n} {category}", "Generate {n} random {category}"])
            n = self.rng.randint(3, 7)
            category = self.rng.choice(CATEGORIES)
            return template.format(n=n, category=category)
        else:
            template = self.rng.choice([
                "Convert to uppercase: {text}",
                "Convert to lowercase: {text}",
                "Reverse order: {text}",
                "Sort alphabetically: {text}",
                "Count items: {text}"
            ])
            return template.format(text=context)
    
    def format_prompt(self, rule: str, enable_thinking: bool = False) -> str:
        messages = [
            {"role": "system", "content": "You are a helpful assistant that follows rules precisely."},
            {"role": "user", "content": rule}
        ]
        if hasattr(tokenizer, 'apply_chat_template'):
            return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=enable_thinking)
        return f"System: You are a helpful assistant.\n\nUser: {rule}\n\nAssistant:"

env = RuleBasedEnvironment(seed=config.seed)
print("✓ Environment created")
print(f"Example: {env.generate_rule()}")

## 📊 Data Generation

In [ ]:
class RuleDataset(Dataset):
    def __init__(self, num_samples: int, environment, enable_thinking: bool = False, seed: int = 42):
        self.environment = environment
        self.enable_thinking = enable_thinking
        rng = random.Random(seed)
        self.prompts = []
        self.rules = []
        for _ in range(num_samples):
            context = None
            if rng.random() > 0.5:
                context = ", ".join([f"item{i}" for i in range(rng.randint(3, 6))])
            rule = environment.generate_rule(context=context)
            prompt = environment.format_prompt(rule, enable_thinking=enable_thinking)
            self.rules.append(rule)
            self.prompts.append(prompt)
    
    def __len__(self):
        return len(self.prompts)
    
    def __getitem__(self, idx):
        return {'prompt': self.prompts[idx], 'rule': self.rules[idx]}

train_dataset = RuleDataset(config.num_train_samples, env, config.enable_thinking, config.seed)
val_dataset = RuleDataset(config.num_val_samples, env, config.enable_thinking, config.seed + 1000)
print(f"✓ Train: {len(train_dataset)}, Val: {len(val_dataset)}")

## 🎁 Reward Function

**TODO: IMPLEMENT YOUR ACTUAL REWARD LOGIC**

In [ ]:
def calculate_reward(prompt: str, response: str, rule: str) -> float:
    """
    Calculate reward - PLACEHOLDER IMPLEMENTATION
    TODO: Implement your actual reward function here!
    """
    reward = 0.0
    words = response.split()
    
    if 5 <= len(words) <= 100:
        reward += 0.3
    if response.strip():
        reward += 0.2
    if "uppercase" in rule.lower() and any(c.isupper() for c in response):
        reward += 0.5
    elif "lowercase" in rule.lower() and any(c.islower() for c in response):
        reward += 0.5
    elif "list" in rule.lower() and "," in response:
        reward += 0.5
    elif "count" in rule.lower() and any(c.isdigit() for c in response):
        reward += 0.5
    
    return reward

print("⚠ WARNING: Placeholder reward function!")
print("  MUST implement calculate_reward() for your task")

## 🧠 Thinking Mode Utilities

In [ ]:
QWEN_THINK_END_TOKEN = 151668

def parse_thinking_response(output_ids, tokenizer) -> Tuple[str, str]:
    if isinstance(output_ids, torch.Tensor):
        output_ids = output_ids.tolist()
    try:
        index = len(output_ids) - output_ids[::-1].index(QWEN_THINK_END_TOKEN)
    except ValueError:
        index = 0
    thinking = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip()
    response = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip()
    return thinking, response

def get_generation_config(config, mode="train") -> Dict:
    temp = config.thinking_temperature if config.enable_thinking else config.temperature
    top_p = config.thinking_top_p if config.enable_thinking else config.top_p
    return {
        "max_new_tokens": config.max_new_tokens,
        "do_sample": True,
        "temperature": temp,
        "top_p": top_p,
        "top_k": config.top_k,
        "pad_token_id": tokenizer.pad_token_id,
        "eos_token_id": tokenizer.eos_token_id,
    }

print("✓ Thinking utilities defined")

## ✅ Baseline Evaluation

In [ ]:
def evaluate_model(model, dataset, num_samples=None, batch_size=8, desc="Eval"):
    model.eval()
    model.config.use_cache = True
    if num_samples is None:
        num_samples = len(dataset)
    else:
        num_samples = min(num_samples, len(dataset))
    
    gen_config = get_generation_config(config, mode="eval")
    gen_config["use_cache"] = True
    total_reward = 0.0
    all_rewards = []
    
    with torch.no_grad():
        for i in tqdm(range(0, num_samples, batch_size), desc=desc):
            batch_end = min(i + batch_size, num_samples)
            batch = [dataset[j] for j in range(i, batch_end)]
            prompts = [item['prompt'] for item in batch]
            rules = [item['rule'] for item in batch]
            inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=config.max_prompt_length).to(DEVICE)
            outputs = model.generate(**inputs, **gen_config)
            generated_ids = outputs[:, inputs.input_ids.size(1):]
            for gen_ids, prompt, rule in zip(generated_ids, prompts, rules):
                if config.enable_thinking and config.parse_thinking:
                    _, response = parse_thinking_response(gen_ids, tokenizer)
                else:
                    response = tokenizer.decode(gen_ids, skip_special_tokens=True)
                reward = calculate_reward(prompt, response, rule)
                all_rewards.append(reward)
                total_reward += reward
    
    model.train()
    return {"mean_reward": total_reward / num_samples, "std_reward": np.std(all_rewards), "num_samples": num_samples}

print("✓ Evaluation function defined")

In [ ]:
print("="*80)
print("BASELINE EVALUATION")
print("="*80)
baseline_metrics = evaluate_model(policy_model, val_dataset, num_samples=min(100, len(val_dataset)), desc="Baseline")
print(f"Baseline: {baseline_metrics['mean_reward']:.4f} ± {baseline_metrics['std_reward']:.4f}")
with open(os.path.join(config.output_dir, "baseline.json"), "w") as f:
    json.dump(baseline_metrics, f, indent=2)
print("="*80)

## 🔧 PPO Utilities

In [ ]:
def compute_log_probs(model, input_ids, attention_mask, return_values=False):
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=return_values)
    logits = outputs.logits
    log_probs = F.log_softmax(logits[:, :-1, :], dim=-1)
    token_log_probs = torch.gather(log_probs, dim=2, index=input_ids[:, 1:].unsqueeze(-1)).squeeze(-1)
    mask = attention_mask[:, 1:].bool()
    token_log_probs = token_log_probs * mask
    if return_values:
        hidden_states = outputs.hidden_states[-1]
        values = value_head(hidden_states).squeeze(-1)
        return token_log_probs, values
    return token_log_probs

def compute_advantages(rewards, values, masks, gamma=1.0, gae_lambda=0.95):
    batch_size, seq_len = rewards.shape
    advantages = torch.zeros_like(rewards)
    gae = 0
    for t in reversed(range(seq_len)):
        next_value = 0 if t == seq_len - 1 else values[:, t + 1]
        delta = rewards[:, t] + gamma * next_value * masks[:, t] - values[:, t]
        gae = delta + gamma * gae_lambda * masks[:, t] * gae
        advantages[:, t] = gae
    returns = advantages + values
    return advantages, returns

def whiten(values, mask, shift_mean=True):
    mean = (values * mask).sum() / mask.sum()
    var = ((values - mean) ** 2 * mask).sum() / mask.sum()
    std = torch.sqrt(var + 1e-8)
    return (values - mean) / std if shift_mean else values / std

print("✓ PPO utilities defined")

## 🚀 Training Setup

In [ ]:
optimizer = torch.optim.AdamW(
    list(policy_model.parameters()) + list(value_head.parameters()),
    lr=config.learning_rate,
    weight_decay=config.weight_decay
)
total_steps = config.num_epochs * config.num_steps_per_epoch
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=config.warmup_steps, num_training_steps=total_steps)
print(f"✓ Optimizer: LR={config.learning_rate}, Steps={total_steps}")

In [ ]:
wandb_run = wandb.init(project=config.wandb_project, name=config.wandb_run_name, config=asdict(config))
print(f"✓ WandB: {wandb_run.get_url()}")

## 🎯 PPO Training Loop

In [ ]:
print("="*80)
print("STARTING CUSTOM PPO TRAINING")
print("="*80)

policy_model.train()
value_head.train()
gen_config = get_generation_config(config, mode="train")
global_step = 0
best_val_reward = -float('inf')
train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)

for epoch in range(config.num_epochs):
    print(f"\nEpoch {epoch+1}/{config.num_epochs}")
    epoch_iter = iter(train_dataloader)
    
    for step in tqdm(range(config.num_steps_per_epoch), desc=f"Epoch {epoch+1}"):
        try:
            batch = next(epoch_iter)
        except StopIteration:
            epoch_iter = iter(train_dataloader)
            batch = next(epoch_iter)
        
        prompts = batch['prompt']
        rules = batch['rule']
        
        # ROLLOUT
        policy_model.eval()
        policy_model.config.use_cache = True
        with torch.no_grad():
            prompt_inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=config.max_prompt_length).to(DEVICE)
            all_responses = []
            all_full_ids = []
            all_masks = []
            for _ in range(config.samples_per_prompt):
                outputs = policy_model.generate(**prompt_inputs, **gen_config)
                generated_ids = outputs[:, prompt_inputs.input_ids.size(1):]
                responses = []
                for gen_ids in generated_ids:
                    if config.enable_thinking and config.parse_thinking:
                        _, resp = parse_thinking_response(gen_ids, tokenizer)
                    else:
                        resp = tokenizer.decode(gen_ids, skip_special_tokens=True)
                    responses.append(resp)
                all_responses.extend(responses)
                all_full_ids.append(outputs)
                mask = torch.ones_like(outputs)
                mask[outputs == tokenizer.pad_token_id] = 0
                all_masks.append(mask)
            all_full_ids = torch.cat(all_full_ids, dim=0)
            all_masks = torch.cat(all_masks, dim=0)
            expanded_prompts = prompts * config.samples_per_prompt
            expanded_rules = rules * config.samples_per_prompt
        
        # REWARDS
        rewards = torch.tensor([calculate_reward(p, r, rule) for p, r, rule in zip(expanded_prompts, all_responses, expanded_rules)], device=DEVICE, dtype=dtype)
        mean_reward = rewards.mean().item()
        
        # OLD PROBS & VALUES
        with torch.no_grad():
            old_log_probs, old_values = compute_log_probs(policy_model, all_full_ids, all_masks, return_values=True)
            ref_log_probs = compute_log_probs(reference_model, all_full_ids, all_masks)
            prompt_len = prompt_inputs.input_ids.size(1)
            old_values_gen = old_values[:, prompt_len:]
            generated_ids_all = all_full_ids[:, prompt_len:]
        
        # ADVANTAGES
        response_mask = (generated_ids_all != tokenizer.pad_token_id).float()
        reward_per_token = torch.zeros_like(generated_ids_all, dtype=dtype)
        for i, reward in enumerate(rewards):
            valid = generated_ids_all[i] != tokenizer.pad_token_id
            reward_per_token[i][valid] = reward / valid.sum().clamp(min=1)
        advantages, returns = compute_advantages(reward_per_token, old_values_gen, response_mask, config.gamma, config.gae_lambda)
        if config.normalize_advantages:
            advantages = whiten(advantages, response_mask)
        
        # PPO UPDATES (Fixed: update after each epoch, not accumulated)
        policy_model.train()
        policy_model.config.use_cache = False
        for ppo_epoch in range(config.ppo_epochs):
            curr_log_probs, curr_values = compute_log_probs(policy_model, all_full_ids, all_masks, return_values=True)
            curr_values_gen = curr_values[:, prompt_len:]
            curr_lp_gen = curr_log_probs[:, prompt_len-1:]
            old_lp_gen = old_log_probs[:, prompt_len-1:]
            ref_lp_gen = ref_log_probs[:, prompt_len-1:]
            
            ratio = torch.exp(curr_lp_gen - old_lp_gen.detach())
            policy_loss = torch.max(
                -advantages.detach() * ratio,
                -advantages.detach() * torch.clamp(ratio, 1-config.clip_range, 1+config.clip_range)
            )
            policy_loss = (policy_loss * response_mask).sum() / response_mask.sum()
            value_loss = ((curr_values_gen - returns.detach())**2 * response_mask).sum() / response_mask.sum()
            kl_penalty = ((curr_lp_gen - ref_lp_gen.detach()) * response_mask).sum() / response_mask.sum()
            
            # Total loss (removed entropy term - was incorrectly calculated)
            loss = policy_loss + config.value_loss_coef * value_loss + config.kl_coef * kl_penalty
            
            # Backward and update after EACH PPO epoch
            loss.backward()
            torch.nn.utils.clip_grad_norm_(list(policy_model.parameters()) + list(value_head.parameters()), config.max_grad_norm)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        # LOGGING
        if global_step % config.log_interval == 0:
            wandb.log({"step": global_step, "loss": loss.item(), "reward": mean_reward, "kl": kl_penalty.item()}, step=global_step)
        
        # EVAL
        if global_step % config.eval_interval == 0 and global_step > 0:
            val_metrics = evaluate_model(policy_model, val_dataset, num_samples=50, desc=f"Eval@{global_step}")
            wandb.log({"val_reward": val_metrics['mean_reward']}, step=global_step)
            if val_metrics['mean_reward'] > best_val_reward:
                best_val_reward = val_metrics['mean_reward']
                best_dir = os.path.join(config.output_dir, "best_model")
                os.makedirs(best_dir, exist_ok=True)
                policy_model.save_pretrained(best_dir)
                tokenizer.save_pretrained(best_dir)
                print(f"\n✓ Best: {best_val_reward:.4f}")
        
        # CHECKPOINT
        if global_step % config.save_interval == 0 and global_step > 0:
            ckpt_dir = os.path.join(config.output_dir, f"checkpoint-{global_step}")
            os.makedirs(ckpt_dir, exist_ok=True)
            policy_model.save_pretrained(ckpt_dir)
            tokenizer.save_pretrained(ckpt_dir)
        
        global_step += 1
        torch.cuda.empty_cache()

print("\n" + "="*80)
print("TRAINING COMPLETE")
print(f"Best val reward: {best_val_reward:.4f}")
print("="*80)

## 📊 Final Evaluation

In [ ]:
print("="*80)
print("FINAL EVALUATION")
print("="*80)
final_metrics = evaluate_model(policy_model, val_dataset, num_samples=len(val_dataset), desc="Final")
print(f"Final: {final_metrics['mean_reward']:.4f} ± {final_metrics['std_reward']:.4f}")
print(f"Improvement: {final_metrics['mean_reward'] - baseline_metrics['mean_reward']:.4f}")
with open(os.path.join(config.output_dir, "final.json"), "w") as f:
    json.dump(final_metrics, f, indent=2)
print("="*80)

## 💾 Save Model

In [ ]:
final_dir = os.path.join(config.output_dir, "final_model")
os.makedirs(final_dir, exist_ok=True)
policy_model.save_pretrained(final_dir)
tokenizer.save_pretrained(final_dir)
print(f"✓ Saved to {final_dir}")

summary = {
    "baseline": baseline_metrics['mean_reward'],
    "final": final_metrics['mean_reward'],
    "best_val": best_val_reward,
    "improvement": final_metrics['mean_reward'] - baseline_metrics['mean_reward']
}
with open(os.path.join(config.output_dir, "summary.json"), "w") as f:
    json.dump(summary, f, indent=2)
print(f"\nSummary: {summary}")
wandb.finish()